# CAPSTONE TWO: NEGATIVE INCOME TAX EXPERIMENTS

## IMPORTS

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
pd.options.display.width=None
pd.options.display.max_colwidth=10000
pd.options.display.max_columns = 10000
pd.options.display.max_rows = 100

In [62]:
gary_df = pd.read_csv('../data/raw/raw_gary_df.csv', dtype=str)
gary_df.drop('Unnamed: 0', axis=1, inplace=True)
gary_df.dropna(inplace=True)
static_df = gary_df.loc[:, [col for col in gary_df.columns if "-" not in col]]

### WIDE DATA -> LONG DATA

In [3]:
def chunkstring(string, length):
    return [string[0+i:length+i] for i in range(0, len(string), length)]
def stub_maker(df, list_of_lengths):
    col_dict = {}
    stub_dict = {}
    for length in list_of_lengths:
        col_dict[length] = [col for col in df.columns if length in col]
    for length in col_dict:
        stub_cols = []
        for column in col_dict[length]:
            numeric_length = int(length[1:])
            if numeric_length == 73:
                numeric_length = 5
            stub = column.split('-')[0][:-1]
            stub_cols.append(stub)
            leng = int(df[column].str.len().unique() / numeric_length)
            df[column] = df[column].apply(chunkstring, args=[leng] )
        stub_dict[length] = stub_cols
    return stub_dict, col_dict
lengths = ['-48', '-42', '-9', '-16', '-73']


In [4]:
stubs, cols = stub_maker(gary_df, lengths)
gary_df

,FAMNUM,PERNUM,STATUS1-48,MOPRES1-48,MRST1-48,RELCODE1-48,DOB,SEX,ENRLDATE,ATTDATE,DENRLDAT,GRADE71,SCHTYP71,GRADE72,SCHTYP72,GRADE73,SCHTYP73,EVEREMP,FAMBUS,YRSTWRK,YRFTWRK,JOBTOTAL,JOBS5YRS,JOBS3YRS,JOBS1YR,INCOME69,EMPLOYED,LABRSTAT,MONTHS,INDUSTRY,OCCUPA,HRSVARY,SEASON,RCNTHRS,RCNTOT,RCNTWAGR,RCNTOTR,RCNTPAY,NRMLHRS,NRMLOT,NRMLWAGR,NRMLOTR,NRMLPAY,DAYHRSR,DAYRATER,DAYPAYR,DAYHRSN,DAYRATEN,DAYPAYN,MOSNOWRK,PLANWORK,SSI1-48,TTI1-48,SS1-48,VA1-48,MISINC1-48,OTHINC1-48,JOBSINC1-48,DAYINC1-48,OJINC1-48,UEMBEN1-48,STRKWC1-48,EMPSTAT1-9,TYPWRKR1-9,WYNOWRK1-9,HRSWEEK1-9,OTHRSWK1-9,HRSREG1-9,OTHREG1-9,DSABLED1-9,DISLMIT1-9,TYPWRK1-48,PCTEMP1-48,PCTWRK1-48,REASON1-48,PCNTLF1-48,PCTUMP1-48,EMPSTA1-48,DISABL1-48,NUMJBS1-48,EXPER1-48,INDUS1-48,OCCUP1-48,WKRLNG1-48,HOURS1-48,HRCHNG1-48,OTHRS1-48,WGRATE1-48,WGCHNG1-48,SINWGS69-73,SJTWGS69-73,WAGES70,QTRWRK70,WGSECON,WGSART1-16,UNEMP1-48,OTRATE1-48,GWAGES0-42,FTXWTH0-42,STXWTH0-42,FICA0-42,JOBS0-42,HRSWRK0-42,REGHRS0-42,OVRTME0-42,SPECL0-42,SPCLCD0-42,AFDC1970,AFDCM070,AFDCBASE,AFDC1-48,AFDCEL1-48,SSI1970,SSIMOS70,SSIBASE,POVLEV,TREATLEV
17,5000,500001,"[00, 00, 00, 00, 00, 00, 02, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04]","[9, 9, 9, 9, 9, 9, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[97, 97, 97, 97, 97, 97, 03, 03, 03, 03, 03, 03, 03, 06, 06, 06, 06, 06, 06, 06, 06, 06, 06, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04, 04]","[97, 97, 97, 97, 97, 97, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01, 01]",5204,2,71189,00000,74196,94,09,09,09,09,09,93,93,93,93,93,93,93,93,99993,93,93,993,993,993,93,93,993,93,9993,9993,999993,993,93,9993,9993,999993,993,9993,999993,993,9993,999993,93,93,"[9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 9997, 9997, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000]","[9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 9997, 9997, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000]","[9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000]","[9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 9997, 9997, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000]","[9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 0000, 0000, 0000, 0100, 0040, 0040, 0040, 0040, 0080, 0080, 0080, 0080, 0080, 0080, 0080, 9997, 9997, 0000, 0000, 0000, 0000, 0000, 0080, 0080, 0080, 0080, 0080, 0000, 0000, 0000, 0000]","[9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997, 0010, 0010, 0010, 0010, 0125, 0125, 0125, 0125, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 9997, 9997, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000]","[9994, 9994, 9994, 9994, 9994, 9994, 9994, 9994, 9994, 9994, 9994, 9994, 9994, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0000, 0

In [5]:
tiles = list(np.arange(1, 49))*5326
test_df = gary_df.copy()
#test_df.assign(period48 = test_tile)
test_dict = {}
test_dict['-48'] = test_df.loc[:, [col for col in test_df if '48' in col]].apply(lambda x: x.explode())
test_dict['-48']['PERIOD'] = tiles
test_dict['-48']

,STATUS1-48,MOPRES1-48,MRST1-48,RELCODE1-48,SSI1-48,TTI1-48,SS1-48,VA1-48,MISINC1-48,OTHINC1-48,JOBSINC1-48,DAYINC1-48,OJINC1-48,UEMBEN1-48,STRKWC1-48,TYPWRK1-48,PCTEMP1-48,PCTWRK1-48,REASON1-48,PCNTLF1-48,PCTUMP1-48,EMPSTA1-48,DISABL1-48,NUMJBS1-48,EXPER1-48,INDUS1-48,OCCUP1-48,WKRLNG1-48,HOURS1-48,HRCHNG1-48,OTHRS1-48,WGRATE1-48,WGCHNG1-48,UNEMP1-48,OTRATE1-48,AFDC1-48,AFDCEL1-48,PERIOD
17,00,9,97,97,9997,9997,9997,9997,9997,9997,9994,0000,9994,0000,0000,00,9994,9994,94,9994,9994,94,94,93,993,993,9993,9993,9993,93,9993,9993,93,9997,93,9994,9994,1
17,00,9,97,97,9997,9997,9997,9997,9997,9997,9994,0000,9994,0000,0000,00,9994,9994,94,9994,9994,94,94,93,993,993,9993,9993,9993,93,9993,9993,93,9997,93,0000,0000,2
17,00,9,97,97,9997,9997,9997,9997,9997,9997,9994,0000,9994,0000,0000,00,9994,9994,94,9994,9994,94,94,93,993,993,9993,9993,9993,93,9993,9993,93,9997,93,0000,0000,3
17,00,9,97,97,9997,9997,9997,9997,9997,9997,9994,0000,9994,0000,0000,00,9994,9994,94,9994,9994,94,94,93,993,993,9993,9993,9993,93,9993,9993,93,9997,93,0100,0100,4
17,00,9,97,97,9997,9997,9997,9997,9997,9997,9994,0000,9994,0000,0000,00,9994,9994,94,9994,9994,94,94,93,993,993,9993,9993,9993,93,9993,9993,93,9997,93,0130,0130,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5342,00,9,94,94,0000,0000,0000,0096,0000,0000,0000,0000,0000,0000,0000,00,0000,0000,94,1000,0000,02,96,00,000,000,0000,0000,0000,00,0000,0000,00,0000,00,0091,0091,44
5342,00,9,94,94,0000,0000,0000,0096,0000,0000,0000,0000,0000,0000,0000,00,0000,0000,94,1000,0000,02,96,00,000,000,0000,0000,0000,00,0000,0000,00,9997,00,0091,0091,45
5342,00,9,94,94,0000,0000,0000,0096,0000,0000,0000,0000,0000,0000,0000,00,0000,0000,94,1000,0000,02,96,00,000,000,0000,0000,0000,00,0000,0000,00,9997,00,0091,0091,46
5342,00,9,94,94,0000,0000,0000,0096,0000,0000,0000,0000,0000,0000,0000,00,0000,0000,94,1000,0000,02,96,00,000,000,0000,0000,0000,00,0000,0000,00,9997,00,0091,0091,47


In [6]:
tiles = list(np.arange(0, 43))*5326
test_df = gary_df.copy()

test_dict['-42'] = test_df.loc[:, [col for col in test_df if '42' in col]].apply(lambda x: x.explode())
print(len(tiles), len(test_dict['-42']))
test_dict['-42']['PERIOD'] = tiles
test_dict['-42']

229018 229018


,GWAGES0-42,FTXWTH0-42,STXWTH0-42,FICA0-42,JOBS0-42,HRSWRK0-42,REGHRS0-42,OVRTME0-42,SPECL0-42,SPCLCD0-42,PERIOD
17,9997,9997,9997,9997,,97,9997,9997,9997,97,0
17,9997,9997,9997,9997,,97,9997,9997,9997,97,1
17,9997,9997,9997,9997,,97,9997,9997,9997,97,2
17,9997,9997,9997,9997,,97,9997,9997,9997,97,3
17,9997,9997,9997,9997,,97,9997,9997,9997,97,4
...,...,...,...,...,...,...,...,...,...,...,...
5342,0000,0000,0000,0000,,00,0000,0000,0000,00,38
5342,0000,0000,0000,0000,,00,0000,0000,0000,00,39
5342,0000,0000,0000,0000,,00,0000,0000,0000,00,40
5342,0000,0000,0000,0000,,00,0000,0000,0000,00,41


In [7]:
tiles = list(np.arange(8, 49, 5))*5326
test_df = gary_df.copy()

test_dict['-9'] = test_df.loc[:, [col for col in test_df if '-9' in col]].apply(lambda x: x.explode())
print(len(tiles), len(test_dict['-9']))
test_dict['-9']['PERIOD'] = tiles
test_dict['-9']

47934 47934


,EMPSTAT1-9,TYPWRKR1-9,WYNOWRK1-9,HRSWEEK1-9,OTHRSWK1-9,HRSREG1-9,OTHREG1-9,DSABLED1-9,DISLMIT1-9,PERIOD
17,94,94,94,994,994,997,997,97,97,8
17,02,94,01,000,000,997,997,97,97,13
17,02,94,06,000,000,997,997,97,97,18
17,02,94,06,000,000,000,000,97,97,23
17,02,94,04,000,000,000,000,94,94,28
...,...,...,...,...,...,...,...,...,...,...
5342,93,93,93,993,993,993,993,93,93,28
5342,93,93,93,993,993,993,993,93,93,33
5342,93,93,93,993,993,993,993,93,93,38
5342,93,93,93,993,993,993,993,93,93,43


In [8]:
tiles = list(np.arange(3, 49, 3))*5326
test_df = gary_df.copy()

test_dict['-16'] = test_df.loc[:, [col for col in test_df if '16' in col]].apply(lambda x: x.explode())
test_dict['-16']['PERIOD'] = tiles
test_dict['-16']

,WGSART1-16,PERIOD
17,0000,3
17,0000,6
17,0000,9
17,0000,12
17,0000,15
...,...,...
5342,0000,36
5342,0000,39
5342,0000,42
5342,0000,45


In [9]:
tiles = list(np.arange(1, 6))*5326
test_df = gary_df.copy()

test_dict['-73'] = test_df.loc[:, [col for col in test_df if '69-73' in col]].apply(lambda x: x.explode())
test_dict['-73']['PERIOD'] = tiles
test_dict['-73']

,SINWGS69-73,SJTWGS69-73,PERIOD
17,999993,999993,1
17,999993,999993,2
17,999993,999993,3
17,999993,999993,4
17,999993,999993,5
...,...,...,...
5342,999993,999993,1
5342,999993,999993,2
5342,999993,999993,3
5342,999993,999993,4


In [10]:
list_of_frames = [test_dict['-42'].reset_index().set_index(['index', 'PERIOD']), 
                  test_dict['-9'].reset_index().set_index(['index', 'PERIOD']),
                  test_dict['-16'].reset_index().set_index(['index', 'PERIOD']),
                  test_dict['-73'].reset_index().set_index(['index', 'PERIOD'])]
final_frame = test_dict['-48'].reset_index().set_index(['index', 'PERIOD']).join(list_of_frames, how='outer')
# final_frame = test_dict['-42'].reset_index().set_index(['index', 'PERIOD']).join(test_dict['-48'].reset_index().set_index(['index', 'PERIOD']), how='outer')
# final_frame.join(test_dict['-9'].reset_index().set_index(['index', 'PERIOD']), how='left')
# final_frame.join(test_dict['-16'].reset_index().set_index(['index', 'PERIOD']), how='left')                 
# final_frame.join(test_dict['-73'].reset_index().set_index(['index', 'PERIOD']), how='left')
final_frame.info
final_frame.to_csv("../data/raw/gary_exploded.csv")

In [11]:
final_frame

STATUS1-48 MOPRES1-48 MRST1-48 RELCODE1-48 SSI1-48 TTI1-48  \
index PERIOD                                                              
17    0             NaN        NaN      NaN         NaN     NaN     NaN   
      1              00          9       97          97    9997    9997   
      2              00          9       97          97    9997    9997   
      3              00          9       97          97    9997    9997   
      4              00          9       97          97    9997    9997   
...                 ...        ...      ...         ...     ...     ...   
5342  44             00          9       94          94    0000    0000   
      45             00          9       94          94    0000    0000   
      46             00          9       94          94    0000    0000   
      47             00          9       94          94    0000    0000   
      48             00          9       94          94    0000    0000   

             SS1-48 VA1-48 MISINC1-48 OTHINC1-48 JOBSINC1-48 DAYINC1-48  \
index PERIOD                                                              
17    0         NaN    NaN        NaN        NaN         NaN        NaN   
      1        9997   9997       9997       9997        9994       0000   
      2        9997   9997       9997       9997        9994       0000   
      3        9997   9997       9997       9997        9994       0000   
      4        9997   9997       9997       9997        9994       0000   
...             ...    ...        ...        ...         ...        ...   
5342  44       0000   0096       0000       0000        0000       0000   
      45       0000   0096       0000       0000        0000       0000   
      46       0000   0096       0000       0000        0000       0000   
      47       0000   0096       0000       0000        0000       0000   
      48       0000   0096       0000       0000        0000       0000   

             OJINC1-48 UEMBEN1-48 STRKWC1-48 TYPWRK1-48 PCTEMP1-48 PCTWRK1-48  \
index PERIOD                                                                    
17    0            NaN        NaN        NaN        NaN        NaN        NaN   
      1           9994       0000       0000         00       9994       9994   
      2           9994       0000       0000         00       9994       9994   
      3           9994       0000       0000         00       9994       9994   
      4           9994       0000       0000         00       9994       9994   
...                ...        ...        ...        ...        ...        ...   
5342  44          0000       0000       0000         00       0000       0000   
      45          0000       0000       0000         00       0000       0000   
      46          0000       0000       0000         00       0000       0000   
      47          0000       0000       0000         00       0000       0000   
      48          0000       0000       0000         00       0000       0000   

             REASON1-48 PCNTLF1-48 PCTUMP1-48 EMPSTA1-48 DISABL1-48  \
index PERIOD                                                          
17    0             NaN        NaN        NaN        NaN        NaN   
      1              94       9994       9994         94         94   
      2              94       9994       9994         94         94   
      3              94       9994       9994         94         94   
      4              94       9994       9994         94         94   
...                 ...        ...        ...        ...        ...   
5342  44             94       1000       0000         02         96   
      45             94       1000       0000         02         96   
      46             94       1000       0000         02         96   
      47             94       1000       0000         02         96   
      48             94       1000       0000         02         96   

             NUMJBS1-48 EXPER1-48 INDUS1-48 OCCUP1-48 WKRLNG1-48 HOURS1-48  \
index

## DATA CLEANING

In [63]:
gary_df = final_frame.fillna(method='ffill')
gary_df.fillna(method="bfill", inplace = True)

In [71]:
gary_df.reset_index(level='PERIOD', inplace=True)

In [73]:
complete_df = static_df.join(gary_df)

In [75]:
complete_df.set_index(['PERNUM', 'PERIOD'])

FAMNUM   DOB SEX ENRLDATE ATTDATE DENRLDAT GRADE71 SCHTYP71  \
PERNUM PERIOD                                                               
500001 0        5000  5204   2    71189   00000    74196      94       09   
       1        5000  5204   2    71189   00000    74196      94       09   
       2        5000  5204   2    71189   00000    74196      94       09   
       3        5000  5204   2    71189   00000    74196      94       09   
       4        5000  5204   2    71189   00000    74196      94       09   
...              ...   ...  ..      ...     ...      ...     ...      ...   
969201 44       9692  2508   2    71179   00000    74196      93       93   
       45       9692  2508   2    71179   00000    74196      93       93   
       46       9692  2508   2    71179   00000    74196      93       93   
       47       9692  2508   2    71179   00000    74196      93       93   
       48       9692  2508   2    71179   00000    74196      93       93   

              GRADE72 SCHTYP72 GRADE73 SCHTYP73 EVEREMP FAMBUS YRSTWRK  \
PERNUM PERIOD                                                            
500001 0           09       09      09       09      93     93      93   
       1           09       09      09       09      93     93      93   
       2           09       09      09       09      93     93      93   
       3           09       09      09       09      93     93      93   
       4           09       09      09       09      93     93      93   
...               ...      ...     ...      ...     ...    ...     ...   
969201 44          93       93      93       93      01     00      64   
       45          93       93      93       93      01     00      64   
       46          93       93      93       93      01     00      64   
       47          93       93      93       93      01     00      64   
       48          93       93      93       93      01     00      64   

              YRFTWRK JOBTOTAL JOBS5YRS JOBS3YRS JOBS1YR INCOME69 EMPLOYED  \
PERNUM PERIOD                                                                
500001 0           93       93       93       93      93    99993       93   
       1           93       93       93       93      93    99993       93   
       2           93       93       93       93      93    99993       93   
       3           93       93       93       93      93    99993       93   
       4           93       93       93       93      93    99993       93   
...               ...      ...      ...      ...     ...      ...      ...   
969201 44          64       03       01       01      01    01440       01   
       45          64       03       01       01      01    01440       01   
       46          64       03       01       01      01    01440       01   
       47          64       03       01       01      01    01440       01   
       48          64       03       01       01      01    01440       01   

              LABRSTAT MONTHS INDUSTRY OCCUPA HRSVARY SEASON RCNTHRS RCNTOT  \
PERNUM PERIOD                                                                 
500001 0            93    993      993    993      93     93     993     93   
       1            93    993      993    993      93     93     993     93   
       2            93    993      993    993      93     93     993     93   
       3            93    993      993    993      93     93     993     93   
       4            93    993      993    993      93     93     993     93   
...                ...    ...      ...    ...     ...    ...     ...    ...   
969201 44           01    072      088    984      00     94     024     00   
       45           01    072      088    984      00     94     024     00   
       46           01    072      088    984      00     94     024     00   
       47           01    072      088    984      00     94     024     00   
       48           01    072      088    984      00     94     024     00   

      

In [76]:
complete_df.replace('93', np.NaN, inplace=True)

In [80]:
complete_df.loc[:,complete_df.isnull().sum() > 1]

,GRADE71,SCHTYP71,GRADE72,SCHTYP72,GRADE73,SCHTYP73,EVEREMP,FAMBUS,YRSTWRK,YRFTWRK,JOBTOTAL,JOBS5YRS,JOBS3YRS,JOBS1YR,EMPLOYED,LABRSTAT,HRSVARY,SEASON,RCNTOT,NRMLOT,MOSNOWRK,PLANWORK,QTRWRK70,AFDCM070,SSIMOS70,TYPWRK1-48,REASON1-48,EMPSTA1-48,DISABL1-48,NUMJBS1-48,HRCHNG1-48,WGCHNG1-48,OTRATE1-48,EMPSTAT1-9,TYPWRKR1-9,WYNOWRK1-9,DSABLED1-9,DISLMIT1-9
17,94,09,09,09,09,09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00,NaN,NaN,00,94,94,94,NaN,NaN,NaN,NaN,94,94,94,97,97
17,94,09,09,09,09,09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00,NaN,NaN,00,94,94,94,NaN,NaN,NaN,NaN,94,94,94,97,97
17,94,09,09,09,09,09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00,NaN,NaN,00,94,94,94,NaN,NaN,NaN,NaN,94,94,94,97,97
17,94,09,09,09,09,09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00,NaN,NaN,00,94,94,94,NaN,NaN,NaN,NaN,94,94,94,97,97
17,94,09,09,09,09,09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00,NaN,NaN,00,94,94,94,NaN,NaN,NaN,NaN,94,94,94,97,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5342,NaN,NaN,NaN,NaN,NaN,NaN,01,00,64,64,03,01,01,01,01,01,00,94,00,00,94,94,00,12,NaN,00,94,02,96,00,00,00,00,NaN,NaN,NaN,NaN,NaN
5342,NaN,NaN,NaN,NaN,NaN,NaN,01,00,64,64,03,01,01,01,01,01,00,94,00,00,94,94,00,12,NaN,00,94,02,96,00,00,00,00,NaN,NaN,NaN,NaN,NaN
5342,NaN,NaN,NaN,NaN,NaN,NaN,01,00,64,64,03,01,01,01,01,01,00,94,00,00,94,94,00,12,NaN,00,94,02,96,00,00,00,00,NaN,NaN,NaN,NaN,NaN
5342,NaN,NaN,NaN,NaN,NaN,NaN,01,00,64,64,03,01,01,01,01,01,00,94,00,00,94,94,00,12,NaN,00,94,02,96,00,00,00,00,NaN,NaN,NaN,NaN,NaN


In [88]:
complete_df.set_index("PERNUM", inplace=True)

In [104]:
complete_df.set_index('PERIOD', append=True, inplace=True)

In [ ]:
def string_stripper(string):
    

In [112]:
complete_df.rename(lambda x:x[:-3] if '-' in x else x, axis = 1, inplace=True)

In [115]:
complete_df

FAMNUM   DOB SEX ENRLDATE ATTDATE DENRLDAT GRADE71 SCHTYP71  \
PERNUM PERIOD                                                               
500001 0        5000  5204   2    71189   00000    74196      94       09   
       1        5000  5204   2    71189   00000    74196      94       09   
       2        5000  5204   2    71189   00000    74196      94       09   
       3        5000  5204   2    71189   00000    74196      94       09   
       4        5000  5204   2    71189   00000    74196      94       09   
...              ...   ...  ..      ...     ...      ...     ...      ...   
969201 44       9692  2508   2    71179   00000    74196     NaN      NaN   
       45       9692  2508   2    71179   00000    74196     NaN      NaN   
       46       9692  2508   2    71179   00000    74196     NaN      NaN   
       47       9692  2508   2    71179   00000    74196     NaN      NaN   
       48       9692  2508   2    71179   00000    74196     NaN      NaN   

              GRADE72 SCHTYP72 GRADE73 SCHTYP73 EVEREMP FAMBUS YRSTWRK  \
PERNUM PERIOD                                                            
500001 0           09       09      09       09     NaN    NaN     NaN   
       1           09       09      09       09     NaN    NaN     NaN   
       2           09       09      09       09     NaN    NaN     NaN   
       3           09       09      09       09     NaN    NaN     NaN   
       4           09       09      09       09     NaN    NaN     NaN   
...               ...      ...     ...      ...     ...    ...     ...   
969201 44         NaN      NaN     NaN      NaN      01     00      64   
       45         NaN      NaN     NaN      NaN      01     00      64   
       46         NaN      NaN     NaN      NaN      01     00      64   
       47         NaN      NaN     NaN      NaN      01     00      64   
       48         NaN      NaN     NaN      NaN      01     00      64   

              YRFTWRK JOBTOTAL JOBS5YRS JOBS3YRS JOBS1YR INCOME69 EMPLOYED  \
PERNUM PERIOD                                                                
500001 0          NaN      NaN      NaN      NaN     NaN    99993      NaN   
       1          NaN      NaN      NaN      NaN     NaN    99993      NaN   
       2          NaN      NaN      NaN      NaN     NaN    99993      NaN   
       3          NaN      NaN      NaN      NaN     NaN    99993      NaN   
       4          NaN      NaN      NaN      NaN     NaN    99993      NaN   
...               ...      ...      ...      ...     ...      ...      ...   
969201 44          64       03       01       01      01    01440       01   
       45          64       03       01       01      01    01440       01   
       46          64       03       01       01      01    01440       01   
       47          64       03       01       01      01    01440       01   
       48          64       03       01       01      01    01440       01   

              LABRSTAT MONTHS INDUSTRY OCCUPA HRSVARY SEASON RCNTHRS RCNTOT  \
PERNUM PERIOD                                                                 
500001 0           NaN    993      993    993     NaN    NaN     993    NaN   
       1           NaN    993      993    993     NaN    NaN     993    NaN   
       2           NaN    993      993    993     NaN    NaN     993    NaN   
       3           NaN    993      993    993     NaN    NaN     993    NaN   
       4           NaN    993      993    993     NaN    NaN     993    NaN   
...                ...    ...      ...    ...     ...    ...     ...    ...   
969201 44           01    072      088    984      00     94     024     00   
       45           01    072      088    984      00     94     024     00   
       46           01    072      088    984      00     94     024     00   
       47           01    072      088    984      00     94     024     00   
       48           01    072      088    984      00     94     024     00   

      

In [121]:
complete_df.drop(complete_df[complete_df['MOPRES1'] == '9'].index, inplace=True)

260,974 - 223,506 = 37,468
Number of rows/months removed because the person was not present in the program, according to the variable MOPRES1.


In [124]:
complete_df.drop(complete_df[complete_df['ATTDATE'] != '00000'].index, inplace=True)

223506 - 204148 = 19,358
Number of rows removed because person attrited from the program before completing the course

In [131]:
complete_df['EMPSTA1'].replace('94', np.NaN, inplace=True)

In [134]:
complete_df['EMPSTA1'].replace('03', '93', inplace=True)

In [135]:
complete_df['EMPSTA1'].unique()

array([nan, '02', '93', '98', '01', '99'], dtype=object)

In [136]:
complete_df.replace('94', np.NaN, inplace=True)

In [141]:
percent_missing = complete_df.isnull().sum() * 100 / len(complete_df)
missing_value_df = pd.DataFrame({'column_name': complete_df.columns,
                                 'percent_missing': percent_missing})
missing_value_df[missing_value_df['percent_missing'] > 0]

,column_name,percent_missing
GRADE71,GRADE71,16.386641
SCHTYP71,SCHTYP71,3.850638
GRADE72,GRADE72,18.468954
SCHTYP72,SCHTYP72,3.850638
GRADE73,GRADE73,35.445853
SCHTYP73,SCHTYP73,3.850638
EVEREMP,EVEREMP,56.701511
FAMBUS,FAMBUS,57.199679
YRSTWRK,YRSTWRK,66.739816
YRFTWRK,YRFTWRK,67.681290


In [15]:
# # tiles = list(np.arange(1, 49))*5326
# test_df = gary_df.copy()
# test_df.assign(period48 = test_tile)
# test_cols_list = [col for col in test_df.columns if '-42' in col or '-9' in col or '-73' in col]
# test_dict = {}
# test_dict['-48'] = test_df.loc[:, [col for col in test_df if '48' in col]].apply(lambda x: x.explode())
# test_dict['-48']['PERIOD-48'] = tiles
# test_dict['-48']

In [16]:
# cols = [col for col in gary_df.columns if '-42' in col]
# for col in cols:
#     lencols = gary_df[col].apply(len) - 43
#     print(lencols.sum())


In [17]:
# def exploder(df, list_of_lengths):
#     exploded_frames = {}
#     for length in list_of_lengths:
#         test_df = df.copy()
#         if length == '-73':
#             num_length = 5
#         elif length =='-42':
#             numeric_length = 43
#         else:
#             num_length = -1 * int(length)
#         values = list(np.arange(1, num_length+1)) * 5326
#         exploded_frames[length] = df.loc[:, [col for col in df if length in col]].apply(lambda x: x.explode())
#         print(values, exploded_frames[length])
#         exploded_frames[length]['PER' + length] = values
#     return exploded_frames
# gary_explode = exploder(gary_df, lengths)

In [18]:
# test_df = gary_df.copy()
# test =  np.arange(1,43,1)
# test_series = np.tile(test, (1,5326))
# test_dict = {}
# test_dict['-42'] = test_df.loc[:, [col for col in test_df.columns if '-42' in col]].apply(lambda x: x.explode())
# len(test_dict['-42'])
# len(test_series[0])

In [19]:
# list_cols = [col for col in test_df.columns if '-48' in col]
# test_dict['-48'] = test_df[list_cols].apply(pd.Series.explode)

# list_cols = [col for col in test_df.columns if '-42' in col]
# test_dict['-42'] = test_df[list_cols].apply(pd.Series.explode)

# list_cols = [col for col in test_df.columns if '-9' in col]
# test_dict['-9'] = test_df[list_cols].apply(pd.Series.explode)

# list_cols = [col for col in test_df.columns if '-16' in col]
# test_dict['-16'] = test_df[list_cols].apply(pd.Series.explode)

# list_cols = [col for col in test_df.columns if '-73' in col]
# test_dict['-73'] = test_df[list_cols].apply(pd.Series.explode)

In [20]:
# test_dict['-48']


In [21]:
# long_df

In [22]:

# for column in wide_cols:
#     if '-48' in column:
#         stubs48_split.append(column.split('-')[0])
#         leng = int(gary_df[column].str.len().unique() /48)
#         gary_df[column] = gary_df[column].apply(chunkstring, args=[leng])
#         for i in range(48):
#             gary_df[column[:-4] + str(i+1)] = gary_df[column].apply(lambda x: x[i])
#     if '-42' in column:
#         stubs42_split.append(column.split('-')[0])
#         leng = int(gary_df[column].str.len().unique() /42)
#         gary_df[column] = gary_df[column].apply(chunkstring, args=[leng])
#         for i in range(42):
#             gary_df[column[:-4] + str(i+1)] = gary_df[column].apply(lambda x: x[i])
#     if '-9' in column:
#         stubs9_split.append(column.split('-')[0])
#         leng = int(gary_df[column].str.len().unique() /9)
#         gary_df[column] = gary_df[column].apply(chunkstring, args=[leng])
#         for i in range(9):
#             gary_df[column[:-3] + str(i+1)] = gary_df[column].apply(lambda x: x[i])  
#     if '-16' in column:
#         stubs16_split.append(column.split('-')[0])
#         leng = int(gary_df[column].str.len().unique() /16)
#         gary_df[column] = gary_df[column].apply(chunkstring, args=[leng])
#         for i in range(16):      
#             gary_df[column[:-3] + str(i+1)] = gary_df[column].apply(lambda x: x[i])
#     if '-73' in column:
#         stubs5_split.append(column.split('-')[0])
#         leng = int(gary_df[column].str.len().unique() /5)
#         gary_df[column] = gary_df[column].apply(chunkstring, args=[leng])
#         for i in range(5):
#             gary_df[column[:-3] + str(i+1)] = gary_df[column].apply(lambda x: x[i])

In [23]:
# gary_df.drop(wide_cols,axis=1, inplace=True)

### HANDLING NULLS

dropping families with no people and people with no families (this was due to a record-keeping error on the part of the experimenters. Families starting with number 4 are supposed to be in the Sacramento file, not Gary).

Converting some of the more-common missing data codes

In [24]:
# gary_df.replace(['9997', '9999','9993','9994', '97', '93'], np.NaN, inplace=True)

dropping the columns with > 75% of their entries being left blank

In [25]:
# def percent_miss(df):
#     # returns the percent of entries that are None in each column.
#    return df.isnull().sum()/df.isnull().count()
# bad_cols = gary_df.loc[:,(percent_miss(gary_df) > 0.75)].columns
# gary_df.drop(bad_cols, axis=1, inplace= True)

ATTDATE stands for attrition date, meaning what date the family left the experiment before it ended. These families left because they either moved away, stopped responding to experimenters, or the active filing member passed. They're being dropped here as we are interested in effects of welfare over time and these cutoff early.

In [26]:
# gary_df = gary_df.loc[gary_df['ATTDATE'] == '00000',:]
# gary_df.drop(['ATTDATE', 'FAMNUM'], axis=1, inplace=True)

## Encoding

Many of the comments within this section are ideas for further analysis, or methods of data cleaning attempted that either failed or were too large of a time sink to complete.

In [27]:
#gary_df = gary_df.loc[gary_df['TREATLEV'] != '0']
#gary_control_df = gary_df.loc[gary_df['TREATLEV'] == '0']

In [28]:
# gary_df.set_index('PERNUM', inplace=True)
# gary_simp_df = pd.get_dummies(gary_df['TREATLEV'], drop_first=True)
# #gary_df.drop(['TREATLEV'], axis=1, inplace=True)
# col_dict = {1:'TREATLEV_1', 2: 'TREATLEV_2', 3:'TREATLEV_3', 4:'TREATLEV_4'}
# gary_simp_df.rename(columns = col_dict, inplace=True)

In [29]:
# gary_simp_df = pd.concat([gary_simp_df, pd.get_dummies(gary_df['POVLEV'], drop_first=True)], axis=1)
# #gary_df.drop(['POVLEV'], axis=1, inplace=True)
# col_dict = {2:'POV_LEV_2', 3: 'POV_LEV_3', 4:'POV_LEV_4', 5:'POV_LEV_5'}
# gary_simp_df.rename(columns = col_dict, inplace=True)

This section of comments was my attempt to take the time-data and parse it out to hopefully generate new rows of data from them. Given more time, I would greatly expand this section, as it has the most potential and would give me the tools to make good features.

In [30]:
# '''def chunkstring(string, length):
#     return [string[0+i:length+i] for i in range(0, len(string), length)]
# for column in gary_df.columns:
#     if '-48' in column:
#         leng = int(gary_df[column].str.len().unique() /48)
#         gary_df[column] = gary_df[column].apply(chunkstring, args=[leng])
#     if '-43' in column:
#         leng = int(gary_df[column].str.len().unique() /43)
#         gary_df[column] = gary_df[column].apply(chunkstring, args=[leng])
#     if '-42' in column:
#         leng = int(gary_df[column].str.len().unique() /42)
#         gary_df[column] = gary_df[column].apply(chunkstring, args=[leng])
#     if '-9' in column:
#         leng = int(gary_df[column].str.len().unique() /9)
#         gary_df[column] = gary_df[column].apply(chunkstring, args=[leng])
#  '''       

In [31]:
# '''
# for column in gary_df.columns:
#     if '-48' in column:
#         leng = int(gary_df[column].str.len().unique() /48)
#         basename = column
#         for i in range(48):
#             gary_df[basename+'-Month'+str(i)] = gary_df[column][i:i+leng]
# gary_df.head()
# '''

Renaming the column for the sake of ease in coding.

In [32]:
# #gary_df.set_index('PERNUM', inplace=True)
# #periodic_columns =['SSI1-48', 'TTI1-48', 'SS1-48', 'VA1-48', 'MISINC1-48',
# #                   'OTHINC1-48', 'JOBINC1-48', 'DAYINC1-48', 'OJINC1-48',
# #                  'UEMBEN1-48', 'STRKWC1-48']
# gary_df.rename(columns= {'EMPSTAT1-9': 'EMPSTAT'}, inplace = True)
# gary_df.info

Ruling out persons for which employment status data was never collected. 

In [33]:
# gary_simp_df['EMPSTAT'] = gary_df.loc[gary_df.EMPSTAT.str.contains('00|01|02', regex=True),'EMPSTAT']

In [34]:
# gary_simp_df.dropna(axis=0, inplace=True)

Unemployed + Actively seeking work, Employed -> in the labor force -> 1   
Unemployed + not actively seeking work -> not in labor force -> 0

In [35]:
# gary_simp_df.loc[(gary_simp_df.EMPSTAT.str.contains('(01)', regex=True)),'EMPSTAT']= '1'
# gary_simp_df.loc[(gary_simp_df.EMPSTAT.str.contains('(00)', regex=True)),'EMPSTAT']= '1'
# #gary_simp_df.loc[(gary_simp_df.EMPSTAT.str.contains('(00)(02)', regex=True)),'EMPSTAT']= 0


In [36]:
# gary_simp_df.loc[gary_simp_df['EMPSTAT'] != '1', 'EMPSTAT'] ='0'

In [37]:
# gary_simp_df

## MODELLING

In [38]:
# X= gary_simp_df.drop('EMPSTAT', axis=1)
# Y= gary_simp_df['EMPSTAT']
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state=2, stratify = Y)
# print(y_train, y_test, X_train, X_test)

## Dummy-test

In [39]:
# dummy = DummyClassifier(strategy = 'most_frequent')
# dummy.fit(X_train, y_train)
# dummy.score(X_test, y_test)

In [40]:
# dummy_report = classification_report(y_test, dummy.predict(X_test), target_names = ['Not in Labor', 'In Labor']
# print(dummy_report)

In [41]:
# lr = LogisticRegression()
# grid = GridSearchCV(estimator=lr,\
#                    param_grid = { \
#                                 'C' : np.arange(0.05, 1.0, .05),\
#                                 'penalty' : ['l2'],\
#                                 'max_iter' : np.arange(500, 5000, 500)},
#                    verbose = 2)
# grid.fit(X_train, y_train)

In [42]:
# print(classification_report(y_test, grid.predict(X_test), target_names=['Not in Labor', 'In Labor']))
# print(confusion_matrix(y_test, grid.predict(X_test)))

**IMPORTANT NOTE:** Logistic Regression does *no* better than guessing the most frequent.

## RANDOM FOREST

In [43]:
# rfc = ensemble.RandomForestClassifier()
# grid = GridSearchCV(estimator = rfc,
#                          param_grid={\
#                                     'max_depth' : [1, 2, 3],\
#                                     'criterion':['gini', 'entropy'],\
#                                     'min_samples_split' : np.arange(0.05,1.0, 0.05),\
#                                              },
#                    verbose=2)
# grid.fit(X_train, y_train)

In [44]:
# print(classification_report(y_test, grid.predict(X_test), target_names=['Not in Labor', 'In Labor']))

**IMPORTANT NOTE** Random Forest doest *no* better than guessing the most frequent

In [45]:
# clf = ensemble.GradientBoostingClassifier()
# rand = RandomizedSearchCV(estimator=clf,\
#                    param_distributions = { \
#                                 'n_estimators' : np.arange(500, 1000, 250),\
#                                 'max_depth' : np.arange(1,3),\
#                                 'learning_rate' : np.arange(0.1, .90, 0.1)})
# rand.fit(X_train, y_train)
# print(classification_report(y_test, rand.predict(X_test), target_names=['Not in Labor', 'In Labor']))

Gradient Boosting does tremendously better at predicting not in the labor force, and thus is the best model of the bunch. 

# FURTHER CONSIDERATIONS

In addition to the comments throughout this notebook better analysis would come from:  
1. Parsing out the 1-48, 1-43, 1-42, 1-9 columns and making them into rows by adding a column for month. The index could then be Person, Month for the data frame. 
2. Using ffill to patch up a lot of the NA's that are either dropped or ignored in this notebook.
3. Running the models with all the features (but not using GridSearch/RandomizedSearch), then doing some basic feature reduction (PCA, etc.)
4. Reconsidering the structure of the categorical data TREATLEV, POVLEV.

In [46]:
#gary_df['NOTINFR'] = gary_df['EMPINT'].str.contains('02')
#gary_df['NOTINFR'] = gary_df['NOTINFR'].astype(int)
#gary_df['EMPGAIN'] = gary_df['EMPINT'].str.contains('(00)(01)', regex=True)

In [47]:
#gary_df['EMPLOSS'] = gary_df['EMPINT'].str.contains('(01)(00)', regex=True)

In [48]:
#gary_df['EMPGAIN'] = gary_df.EMPGAIN.astype(int)

In [49]:
#gary_df['EMPLOSS'] = gary_df.EMPLOSS.astype(int)

In [50]:
#columns_for_later = ['EMPLOSS', 'EMPGAIN']
#gary_df.drop(columns_for_later, axis=1, inplace=True)

In [51]:
#gary_df['EMP'] = gary_df['EMPINT'].str.contains('01')
#gary_df['EMP'] = gary_df.EMP.astype(int)